# Downloading data from the fixed met stations

Seklima (https://seklima.met.no) doe snot allow to download large amount of data, instead, we must use https://frost.met.no. Here, I setup the API and attempt to download the data from the stations nearby Longyearbyen. This code is adapted from https://frost.met.no/python_example.html.

In [1]:
import requests
import pandas as pd

# Insert your own client ID here
client_id = '83df1705-b64e-4a54-a0b9-825064fc860c' # this is a client_id for Clément Cherblanc

In [19]:
# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN99840',#use "XXX,YYY" (comma) for multiple stations, variable, etc.
    'elements': 'wind_from_direction', 
    'referencetime': '2023-01-02/2024-01-01',
    #'referencetime': '1995-01-02/2024-01-01',
}
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()

In [20]:
# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Data retrieved from frost.met.no!


In [26]:
# Initialize an empty list to store rows
rows = []

for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    rows.append(row)  # Append the DataFrame to the list

# Use pd.concat() to merge all rows at once
df = pd.concat(rows, ignore_index=True)

# Reset index (optional, but useful for clean DataFrame)
df = df.reset_index(drop=True)

In [27]:
df.head()

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,wind_from_direction,107,degrees,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT1H,0,C,2,0,2023-02-01T00:00:00.000Z,SN99840:0
1,wind_from_direction,107,degrees,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT10M,0,C,2,0,2023-02-01T00:00:00.000Z,SN99840:0
2,wind_from_direction,106,degrees,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT10M,0,C,2,0,2023-02-01T00:10:00.000Z,SN99840:0
3,wind_from_direction,120,degrees,"{'levelType': 'height_above_ground', 'unit': '...",PT20M,PT30M,0,C,2,2,2023-02-01T00:20:00.000Z,SN99840:0
4,wind_from_direction,103,degrees,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,PT10M,0,C,2,0,2023-02-01T00:20:00.000Z,SN99840:0
